# c-Si panel production

In [11]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as ba
import numpy as np

In [12]:
bd.projects.set_current("ecoinvent-3.11-cutoff-bw25")
bd.projects

Brightway2 projects manager with 7 objects:
	Another one bikes the dust
	Bicycle example
	USEEIO-1.1
	default
	ecoinvent-3.11-cutoff-bw25
	ecoinvent-3.12-cutoff
	electric_vehicle_standalone
Use `projects.report()` to get a report on all projects.

In [15]:
bi.restore_project_directory('/srv/data/brightway2-project-ecoinvent-3.11-cutoff-bw25-backup.tar.gz', overwrite_existing=True)

Restoring project backup archive - this could take a few minutes...
Restored project: ecoinvent-3.11-cutoff-bw25


'ecoinvent-3.11-cutoff-bw25'

In [16]:
# Adding the databases into the PV project
db = bd.Database("ecoinvent-3.11-cutoff")
db.register()

In [17]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

## Creating the nodes

In [19]:
# Creating the nodes for the PV module and PV module production
PV_module = db.new_node(
    name="c-Si PV module",
    unit="kWp",
    type=bd.labels.product_node_default,
)
PV_production = db.new_node(
    name="c-Si PV production",
    location="DE",
    type=bd.labels.process_node_default,
)
PV_EoL = db.new_node(
    name="c-Si PV EOL",
    location="DE",
    type=bd.labels.process_node_default,
)
PV_module.save()
PV_production.save()
PV_EoL.save()

In [20]:
# Getting nodes from ecoinvent
silicon_solar_grade= bd.get_node(id=246559704623259649, database="ecoinvent-3.11-cutoff")
aluminum_frame= bd.get_node(id=246559762810839042, database="ecoinvent-3.11-cutoff")
glass_front_sheet= bd.get_node(id=246559769777577986, database="ecoinvent-3.11-cutoff")
eva_encapsulant= bd.get_node(id=246559786403799041, database="ecoinvent-3.11-cutoff")
backsheet_pet= bd.get_node(id=246559722780401673, database="ecoinvent-3.11-cutoff")
copper_wiring= bd.get_node(id=246559703599849475, database="ecoinvent-3.11-cutoff")
silver_contacts= bd.get_node(id=246559770343809025, database="ecoinvent-3.11-cutoff")
junction_box= bd.get_node(id=246559767613317123, database="ecoinvent-3.11-cutoff")
electricity_mix= bd.get_node(id=246559746184617985, database="ecoinvent-3.11-cutoff")
heat_naturalgas= bd.get_node(id=246559730611167232, database="ecoinvent-3.11-cutoff")
all_transport= bd.get_node(id=246559732137893888, database="ecoinvent-3.11-cutoff")
silicon_kerf_loss= bd.get_node(id=246559763263823876, database="ecoinvent-3.11-cutoff")
waste_plastic = bd.get_node(id=246559782427598848, database="ecoinvent-3.11-cutoff")
waste_others = bd.get_node(id=246559740333563904, database="ecoinvent-3.11-cutoff")

## Creating the edges

In [21]:
import pandas as pd

# Step 1: Read the CSV file
# Replace 'your_file.csv' with your actual filename or path
dfi = pd.read_csv('material_inflows_PV.csv')
dfo = pd.read_csv('material_outflows_PV.csv')
capi = pd.read_csv('stockdriven_inflow_capacity.csv')
capo = pd.read_csv('stockdriven_outflow_capacity.csv')

# Step 2: Check the columns (optional, to confirm structure)
print(dfi.columns)

# Step 3: Convert each column into a separate list
Year = dfi['Year'].tolist()
Ag = dfi['Ag'].tolist()
Al = dfi['Al'].tolist()
Cu = dfi['Cu'].tolist()
Electrolyte = dfi['Electrolyte'].tolist()
Li = dfi['Li'].tolist()
PP = dfi['PP'].tolist()
Si = dfi['Si'].tolist()
Steel = dfi['Steel'].tolist()

Si_waste = dfo['Si'].tolist()
Ag_waste = dfo['Ag'].tolist()
Al_waste = dfo['Al'].tolist()
Cu_waste = dfo['Cu'].tolist()

inflow_capacity = capi['PV'].tolist()   # in kWp
outflow_capacity = capo['PV'].tolist()  # in kWp

inflow = [110, 21, 21, 21, 21, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 16, 16, 16, 16, 126]
inflow_capacity_base = np.array(inflow)*1e6       # in kWp
outflow = [0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	110]
outflow_capacity_base = np.array(outflow)*1e6       # in kWp

Index(['Year', 'Ag', 'Al', 'Cu', 'Electrolyte', 'Li', 'PP', 'Si', 'Steel'], dtype='object')


In [22]:
# Defining the inventory update based on input data from MFA

def update_inventory_manufacturing(Time): 

    time_index = Year.index(Time)
   
    # Main production output
    PV_production.new_edge(
        amount=1,
        input=PV_module,
        type=bd.labels.production_edge_default,
        functional=True,
    ).save()
    
    # Material inputs    
    PV_production.new_edge(
        amount=Si[time_index]/(inflow_capacity[time_index]),
        input=silicon_solar_grade,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()
    PV_production.new_edge(
        amount=Al[time_index]/(inflow_capacity[time_index]),
        input=aluminum_frame,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()    
    PV_production.new_edge(
        amount=66,
        input=glass_front_sheet,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()    
    PV_production.new_edge(
        amount=3.3,
        input=eva_encapsulant,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()    
    PV_production.new_edge(
        amount=2.2,
        input=backsheet_pet,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()    
    PV_production.new_edge(
        amount=Cu[time_index]/(inflow_capacity[time_index]),
        input=copper_wiring,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()    
    PV_production.new_edge(
        amount=Ag[time_index]/(inflow_capacity[time_index]),
        input=silver_contacts,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()    
    PV_production.new_edge(
        amount=1.1/18.5,             # Junction box unit in piece, 18.5 kg/piece
        input=junction_box,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()
    
    # Energy
    PV_production.new_edge(
        amount=2200,
        input=electricity_mix,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()    
    PV_production.new_edge(
        amount=275,
        input=heat_naturalgas,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()
    
    # Transport
    PV_production.new_edge(
        amount=3850,
        input=all_transport,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()

def update_inventory_EoL(Time): 

    time_index = Year.index(Time)
    
    # Waste (manufacturing)
    PV_EoL.new_edge(
        amount=Si_waste[time_index]/(outflow_capacity[time_index]),
        input=silicon_kerf_loss,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()
    PV_EoL.new_edge(
        amount=(Ag_waste[time_index] + Al_waste[time_index] + Cu_waste[time_index])/(outflow_capacity[time_index]),
        input=waste_others,
        type=bd.labels.consumption_edge_default,
        functional=True,
    ).save()


GWP CALCULATIONS

In [23]:
GWP_per_kWp_by_year_manufacturing = {}

for i in Year:
    update_inventory_manufacturing(i)
    functional_unit = PV_module
    method_gwp_100 = (
        'ecoinvent-3.11',
        'CML v4.8 2016 no LT',
        'climate change no LT',
        'global warming potential (GWP100) no LT'
    )
    impact_category=method_gwp_100
    
    lca = bc.LCA({functional_unit: 1}, method=impact_category)
    lca.lci()
    lca.lcia()
    GWP_per_kWp_by_year_manufacturing[i] = lca.score

In [ ]:
GWP_per_kWp_by_year_EoL = {}

for i in Year:
    update_inventory_EoL(i)
    functional_unit = PV_module
    method_gwp_100 = (
        'ecoinvent-3.11',
        'CML v4.8 2016 no LT',
        'climate change no LT',
        'global warming potential (GWP100) no LT'
    )
    impact_category=method_gwp_100
    
    lca = bc.LCA({functional_unit: 1}, method=impact_category)
    lca.lci()
    lca.lcia()
    GWP_per_kWp_by_year_EoL[i] = lca.score

In [ ]:
GWP_per_kWp_by_year_manufacturing

In [ ]:
GWP_per_kWp_by_year_EoL

In [ ]:
GWP_per_kWp_by_year = {key: GWP_per_kWp_by_year_manufacturing[key] + GWP_per_kWp_by_year_EoL.get(key, 0) for key in GWP_per_kWp_by_year_manufacturing}

In [ ]:
GWP_per_kWp_by_year

In [ ]:
GWP_total_by_year_manufacturing = {key: value * factor for key, value, factor in zip(GWP_per_kWp_by_year_manufacturing.keys(), GWP_per_kWp_by_year_manufacturing.values(), inflow_capacity)}
GWP_total_by_year_EoL ={key: value * factor for key, value, factor in zip(GWP_per_kWp_by_year_EoL.keys(), GWP_per_kWp_by_year_EoL.values(), outflow_capacity)}
GWP_total_by_year = {key: GWP_total_by_year_manufacturing[key] + GWP_total_by_year_EoL.get(key, 0) for key in GWP_total_by_year_manufacturing}
GWP_total_by_year

In [ ]:
first_key = next(iter(GWP_per_kWp_by_year_manufacturing))
base1_value_manufacturing = GWP_per_kWp_by_year_manufacturing[first_key]
Base1_GWP_total_by_year_manufacturing = {year: base1_value_manufacturing * factor for year, factor in zip(GWP_per_kWp_by_year_manufacturing.keys(), inflow_capacity_base)}

first_key_EoL = next(iter(GWP_per_kWp_by_year_EoL))
base1_value_EoL = GWP_per_kWp_by_year_EoL[first_key_EoL]
Base1_GWP_total_by_year_EoL = {year: base1_value_EoL * factor for year, factor in zip(GWP_per_kWp_by_year_EoL.keys(), outflow_capacity_base)}

Base1_GWP_total_by_year = {key: Base1_GWP_total_by_year_manufacturing[key] + Base1_GWP_total_by_year_EoL.get(key, 0) for key in Base1_GWP_total_by_year_manufacturing}
Base1_GWP_total_by_year

In [ ]:
first_key = next(iter(GWP_per_kWp_by_year_manufacturing))
base2_value_manufacturing = GWP_per_kWp_by_year_manufacturing[first_key]
Base2_GWP_total_by_year_manufacturing = {year: base2_value_manufacturing * factor for year, factor in zip(GWP_per_kWp_by_year_manufacturing.keys(), inflow_capacity)}

first_key_EoL = next(iter(GWP_per_kWp_by_year_EoL))
base2_value_EoL = GWP_per_kWp_by_year_EoL[first_key_EoL]
Base2_GWP_total_by_year_EoL = {year: base2_value_EoL * factor for year, factor in zip(GWP_per_kWp_by_year_EoL.keys(), outflow_capacity)}

Base2_GWP_total_by_year = {key: Base2_GWP_total_by_year_manufacturing[key] + Base2_GWP_total_by_year_EoL.get(key, 0) for key in Base2_GWP_total_by_year_manufacturing}
Base2_GWP_total_by_year

In [ ]:
Difference_GWP_total_lifetime = {key: Base2_GWP_total_by_year[key] - Base1_GWP_total_by_year.get(key, 0) for key in Base2_GWP_total_by_year}
Difference_GWP_total_material = {key: GWP_total_by_year[key] - Base2_GWP_total_by_year.get(key, 0) for key in GWP_total_by_year}
Difference_GWP_total_overall = {key: GWP_total_by_year[key] - Base1_GWP_total_by_year.get(key, 0) for key in GWP_total_by_year}

In [ ]:
import matplotlib.pyplot as plt

# Extract years and values
years = list(GWP_per_kWp_by_year.keys())

# Bar width and positioning
bar_width = 0.35
x = range(len(years))

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], list(GWP_per_kWp_by_year.values()), width=bar_width)

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq/kWp')
plt.title('GWP100 per kWp per Year')
plt.xticks(x, years, rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
### ALL compared
# Extract values
values1 = list(GWP_total_by_year.values())
values2 = list(Base1_GWP_total_by_year.values())
values3 = list(Base2_GWP_total_by_year.values())


# Bar width and positioning
bar_width = 0.35
x = range(len(years))

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/3 for i in x], values1, width=bar_width, label='Dynamic Lifetime Material Scenario',color ='blue' )
plt.bar([i + bar_width/3 for i in x], values2, width=bar_width, label='Base Static Lifetime', color='orange')
plt.bar([i + 2*bar_width/3 for i in x], values3, width=bar_width, label='Base Dynamic Lifetime', color='red')

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq')
plt.title('GWP100 for all scenarios')
plt.xticks(x, years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

### GRAPH BASE 1 vs BASE 2
# Extract values
values1 = list(Base1_GWP_total_by_year.values())
values2 = list(Base2_GWP_total_by_year.values())

# Bar width and positioning
bar_width = 0.35
x = range(len(years))

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], values1, width=bar_width, label='Base Dynamic Lifetime', color='orange')
plt.bar([i + bar_width/2 for i in x], values2, width=bar_width, label='Base Static Lifetime', color='red')

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq')
plt.title('GWP100 BASE STATIC vs DYNAMIC Lifetime')
plt.xticks(x, years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


### GRAPH DYNAMIC VS BASE 1
# Extract values
values1 = list(GWP_total_by_year.values())

# Second dataset (baseline scenario): I wrote it constant 20000000000 manually for now
values2 = list(Base1_GWP_total_by_year.values())

# Bar width and positioning
bar_width = 0.35
x = range(len(years))

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], values1, width=bar_width, label='Dynamic Lifetime Material Scenario',color ='blue' )
plt.bar([i + bar_width/2 for i in x], values2, width=bar_width, label='Base Dynamic Lifetime', color='orange')

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq')
plt.title('GWP100 DYNAMIC vs BASE STATIC Lifetime')
plt.xticks(x, years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

### GRAPH DYNAMIC VS BASE 2
# Extract values
values1 = list(GWP_total_by_year.values())
values2 = list(Base2_GWP_total_by_year.values())

# Bar width and positioning
bar_width = 0.35
x = range(len(years))

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], values1, width=bar_width, label='Dynamic Lifetime Material Scenario',color ='blue' )
plt.bar([i + bar_width/2 for i in x], values2, width=bar_width, label='Base Static Lifetime', color='red')

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq')
plt.title('GWP100 DYNAMIC vs BASE DYNAMC Lifetime')
plt.xticks(x, years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
Difference_GWP_total_lifetime = {key: Base2_GWP_total_by_year[key] - Base1_GWP_total_by_year.get(key, 0) for key in Base2_GWP_total_by_year}
Difference_GWP_total_material = {key: GWP_total_by_year[key] - Base2_GWP_total_by_year.get(key, 0) for key in GWP_total_by_year}
Difference_GWP_total_overall = {key: GWP_total_by_year[key] - Base1_GWP_total_by_year.get(key, 0) for key in GWP_total_by_year}

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], list(Difference_GWP_total_lifetime.values()), width=bar_width)

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq')
plt.title('GWP100 Difference Between Base static and dynamic lifetime')
plt.xticks(x, years, rotation=45)
plt.tight_layout()
plt.show()

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], list(Difference_GWP_total_material.values()), width=bar_width)

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq')
plt.title('GWP100 Difference Between Dynamic and Base dynamic lifetime')
plt.xticks(x, years, rotation=45)
plt.tight_layout()
plt.show()

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], list(Difference_GWP_total_overall.values()), width=bar_width)

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg CO2-eq')
plt.title('GWP100 Difference Between Dynamic and Base static lifetime')
plt.xticks(x, years, rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], list(ADP_per_kWp_by_year.values()), width=bar_width)

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg SB-eq/kWp')
plt.title('ADP per kWp per Year')
plt.xticks(x, years, rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Extract values
values3 = list(ADP_total_by_year.values())

# Second dataset (baseline scenario): I wrote it constant 20000000000 manually for now
values4 = list(Base_ADP_total_by_year.values())

# Bar width and positioning
bar_width = 0.35
x = range(len(years))

# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], values3, width=bar_width, label='Dynamic Scenario', )
plt.bar([i + bar_width/2 for i in x], values4, width=bar_width, label='Baseline Scenario', color='orange')

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg SB-eq')
plt.title('ADP of the Inflow Capacity')
plt.xticks(x, years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot bars side by side
plt.figure(figsize=(12,6))
plt.bar([i - bar_width/2 for i in x], list(Difference_ADP_total.values()), width=bar_width)

# Labels and title
plt.xlabel('Year')
plt.ylabel('kg Sb-eq')
plt.title('ADP Difference Between Baseline and Dynamic')
plt.xticks(x, years, rotation=45)
plt.tight_layout()
plt.show()